In [3]:
from pathlib import Path
import tqdm
import torch
import pandas as pd
import numpy as np
from diffusers import StableDiffusionPipeline
from transformers import pipeline, set_seed
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import cv2
     

In [ ]:
from diffusers import StableDiffusionPipeline

model_id = "stabilityai/stable-diffusion-2-1-base"
pipeline = StableDiffusionPipeline.from_pretrained(model_id)
pipeline.to("cuda")  # Use "cpu" if you don't have a GPU


c:\Main\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 13 files:  31%|███       | 4/13 [00:10<00:22,  2.52s/it]


c:\Main\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Fetching 13 files:  31%|███       | 4/13 [00:00<00:01,  4.66it/s]


In [ ]:
from diffusers import StableDiffusionPipeline
import torch

# Define the model path (update if needed)
MODEL_PATH = "stabilityai/stable-diffusion-2-1-base"

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model with optimized settings
pipeline = StableDiffusionPipeline.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    use_safetensors=True
).to(device)

# Enable memory-efficient attention (if supported)
try:
    pipeline.enable_xformers_memory_efficient_attention()
except Exception as e:
    print(f"Xformers not available: {e}")

print("Stable Diffusion 2-1 Base Model Loaded Successfully!")


In [3]:
import os
import random
import torch
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from wordcloud import WordCloud
from diffusers import StableDiffusionPipeline
from pytorch_fid import fid_score
from PIL import Image

# Ensure paths exist
image_folder = "dataset/images"  # Update with actual dataset image path
generated_images_folder = "generated_images"
os.makedirs(generated_images_folder, exist_ok=True)

# Load dataset (Assume dataset.csv contains a column 'text' with prompts)
df = pd.read_csv("dataset/dataset.csv")

# 1️⃣ Word Cloud - Visualizing Common Words in Prompts
text_data = " ".join(df['text'].astype(str))
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(text_data)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Most Common Words in Text Prompts")
plt.show()

# 2️⃣ Text Length Distribution - Check Prompt Sizes
df['text_length'] = df['text'].apply(lambda x: len(x.split()))
plt.hist(df['text_length'], bins=20, color="skyblue", edgecolor="black")
plt.xlabel("Text Prompt Length (words)")
plt.ylabel("Frequency")
plt.title("Distribution of Text Prompt Lengths")
plt.show()

# 3️⃣ Random Sample Images from Dataset
sample_images = random.sample(os.listdir(image_folder), 5)

fig, axes = plt.subplots(1, 5, figsize=(15, 5))
for i, img_name in enumerate(sample_images):
    img = cv2.imread(os.path.join(image_folder, img_name))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    axes[i].imshow(img)
    axes[i].axis("off")
    axes[i].set_title(img_name)

plt.suptitle("Random Sample Images from Dataset")
plt.show()

# 4️⃣ Load Stable Diffusion 2-1 Model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1").to(device)

# 5️⃣ Generate Images from Text Prompts
sample_prompts = [
    "A futuristic city skyline at sunset",
    "A cat sitting on a mountain peak",
    "A cyberpunk warrior in a neon-lit street",
    "A spaceship landing on Mars",
    "A medieval castle surrounded by mist"
]

generated_images = []
fig, axes = plt.subplots(1, 5, figsize=(15, 5))
for i, prompt in enumerate(sample_prompts):
    with torch.no_grad():
        generated_image = model(prompt).images[0]
    
    # Save generated image
    image_path = f"{generated_images_folder}/generated_{i}.png"
    generated_image.save(image_path)
    generated_images.append(image_path)

    # Display generated image
    axes[i].imshow(generated_image)
    axes[i].axis("off")
    axes[i].set_title(prompt)

plt.suptitle("Generated Images from Stable Diffusion")
plt.show()

# 6️⃣ Compare Real vs. Generated Images
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

# Load real images
real_images = random.sample(os.listdir(image_folder), 5)
for i, img_name in enumerate(real_images):
    img = cv2.imread(os.path.join(image_folder, img_name))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    axes[0, i].imshow(img)
    axes[0, i].axis("off")
    axes[0, i].set_title("Real Image")

# Load generated images
for i, gen_image_path in enumerate(generated_images):
    img = Image.open(gen_image_path)
    axes[1, i].imshow(img)
    axes[1, i].axis("off")
    axes[1, i].set_title("Generated Image")

plt.suptitle("Comparison: Real vs. Generated Images")
plt.show()

# 7️⃣ Compute FID Score (Evaluate Image Quality)
fid = fid_score.calculate_fid_given_paths([image_folder, generated_images_folder], batch_size=16, device=device)
print(f"FID Score: {fid}")


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/dataset.csv'

In [ ]:
class CFG:
    device = "cuda"
    seed = 42
    generator = torch.Generator(device).manual_seed(seed)
    image_gen_steps = 35
    image_gen_model_id = "stabilityai/stable-diffusion-2"
    image_gen_size = (400,400)
    image_gen_guidance_scale = 9
    prompt_gen_model_id = "gpt2"
    prompt_dataset_size = 6
    prompt_max_length = 12